In [ ]:
import json
import pandas as pd
import os
from snowflake.core import Root
from snowflake.snowpark import Session

# HOL Setup Notebook
Run the following commands to provision the necessary objects for the Snowflake Intelligence HOL

- add snowflake package


## Setup User Schema

In [ ]:
USE ROLE HOL_ADMIN;

SET CURRENT_USER = CURRENT_USER();

USE DATABASE HOL;
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($CURRENT_USER);


In [ ]:
USE SCHEMA IDENTIFIER($CURRENT_USER);

## Setup Unstructured Chunks Table

In [ ]:

-- CREATE OR REPLACE TABLE SEC_FILINGS_CHUNKS (
CREATE TABLE IF NOT EXISTS SEC_FILINGS_CHUNKS (
    document_name varchar,
    chunk_id varchar,
    chunk_text varchar,
    document_date date
);

-- Perform OCR and Chunk Text
INSERT INTO SEC_FILINGS_CHUNKS  
    with doc_text as (
        select
            split_part(relative_path,'.',0) as document_name,
            AI_PARSE_DOCUMENT (
                TO_FILE('@DATA.SEC_FILINGS_STAGE',relative_path),
                {'mode': 'OCR' , 'page_split': false}
            ):content::varchar as doc_text,
            to_date(split_part(document_name,'_',3),'MM-DD-YY') as doc_date
        from directory('@DATA.SEC_FILINGS_STAGE')
    )
    
    , chunked as (
        select 
            document_name,
            SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              doc_text,
              'none',
              500
            ) as chunks,
            doc_date
        from doc_text
    )
    
    , flattened as (
        select
            document_name,
            index as chunk_id,
            value::varchar as chunk_text,
            doc_date
        from chunked,
            TABLE(FLATTEN(INPUT => CHUNKS))
    )
    
    SELECT * FROM flattened
;


In [ ]:
SELECT * FROM SEC_FILINGS_CHUNKS;

## Create Search Service for RAG

In [ ]:

CREATE CORTEX SEARCH SERVICE IF NOT EXISTS SEC_FILINGS_SEARCH
  ON CHUNK_TEXT
  ATTRIBUTES DOCUMENT_DATE
  WAREHOUSE = SNOWFLAKE_LEARNING_WH
  TARGET_LAG = '7 DAYS'
  AS (SELECT * FROM SEC_FILINGS_CHUNKS);

-- RUN A SAMPLE QUERY
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'SEC_FILINGS_SEARCH',
    '{
      "query": "NVIDIA revenue guidance",
      "columns": ["DOCUMENT_NAME", "CHUNK_ID", "DOCUMENT_DATE", "CHUNK_TEXT"],
      "limit": 5
    }'
  )
)['results'] AS results;

## Create a Custom Tool for the Agent
Just a dummy stock price tool, in reality this could easily be replaced with an API call using an external access integration.

In [ ]:
-- Dummy Tool
CREATE OR REPLACE PROCEDURE GET_STOCK_PRICE(TICKER STRING, EXCHANGE STRING)
RETURNS FLOAT
LANGUAGE SQL
AS
$$
DECLARE
    stock_price FLOAT;
BEGIN
    stock_price := 185.88;
    RETURN stock_price;
END;
$$;

call get_stock_price('nvda','nyse');

## Verify Existing Price History Data
For structured data we will be using a preloaded table made available for the user.

In [ ]:
CREATE OR REPLACE TABLE STOCK_PRICE_HISTORY CLONE DATA.STOCK_PRICE_HISTORY;

In [ ]:
SELECT * FROM STOCK_PRICE_HISTORY

## Create the Semantic View

This could also be done a number of different ways, mainly through Snowsight. If you have YAML already provisioned in this case you can simply use the system function as shown below.

NOTE: Make sure to change schema and database in the yaml below before creation to HOL.<YOUR_USER>

In [ ]:
CALL SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML(
  'HOL.' || $CURRENT_USER,
  $$
name: stock_prices_semantic_model
description: Semantic model for historical daily stock prices (OHLCV) with dividends and splits.
custom_instructions: >
  If the user doesn't specify a date filter, apply a default filter for the last year.
  Round numeric outputs to 2 decimal places.
  If the user asks for 'returns' or 'change', interpret as percentage change over
  the selected period (e.g., (last_close - first_close) / first_close * 100).

tables:
  - name: stock_prices
    description: Daily OHLCV per ticker.
    base_table:
      database: HOL
      schema: jacob_scott
      table: STOCK_PRICE_HISTORY

    time_dimensions:
      - name: date
        synonyms: ["trading date", "day"]
        description: Trading day (no time component).
        expr: DATE
        data_type: DATE
        unique: false

    dimensions:
      - name: ticker
        synonyms: ["symbol"]
        description: Stock ticker symbol (e.g., NVDA).
        expr: TICKER
        data_type: TEXT
        unique: false
        is_enum: true

    facts:
      - name: open_price
        synonyms: ["open"]
        description: Opening price for the day.
        expr: OPEN_PRICE
        data_type: NUMBER

      - name: high_price
        synonyms: ["high"]
        description: Intraday high price.
        expr: HIGH_PRICE
        data_type: NUMBER

      - name: low_price
        synonyms: ["low"]
        description: Intraday low price.
        expr: LOW_PRICE
        data_type: NUMBER

      - name: close_price
        synonyms: ["close", "closing price"]
        description: Closing price for the day.
        expr: CLOSE_PRICE
        data_type: NUMBER

      - name: volume
        synonyms: ["shares traded"]
        description: Number of shares traded during the day.
        expr: VOLUME
        data_type: NUMBER

      - name: dividends
        synonyms: ["dividend"]
        description: Cash dividends paid on the day.
        expr: DIVIDENDS
        data_type: NUMBER

      - name: stock_splits
        synonyms: ["split ratio"]
        description: Stock split ratio for the day (e.g., 4.0 == 4-for-1).
        expr: STOCK_SPLITS
        data_type: NUMBER

    metrics:
      - name: avg_close_price
        synonyms: ["average close"]
        description: Average closing price over the selected period.
        expr: AVG(CLOSE_PRICE)

      - name: max_close_price
        synonyms: ["record close", "all-time high close"]
        description: Maximum closing price over the selected period.
        expr: MAX(CLOSE_PRICE)

      - name: min_close_price
        synonyms: ["lowest close"]
        description: Minimum closing price over the selected period.
        expr: MIN(CLOSE_PRICE)

      - name: total_volume
        synonyms: ["sum volume", "trading volume"]
        description: Total shares traded over the selected period.
        expr: SUM(VOLUME)

    filters:
      - name: last_30_days
        synonyms: ["past month"]
        description: Filter to the last 30 calendar days.
        expr: "DATE >= DATEADD('day', -30, CURRENT_DATE())"

      - name: last_year
        synonyms: ["past year", "last 12 months"]
        description: Filter to the last 365 days.
        expr: "DATE >= DATEADD('day', -365, CURRENT_DATE())"

verified_queries:
  - name: average_close_by_month
    question: What was the average closing price by month for NVDA this year?
    sql: >
      SELECT DATE_TRUNC('month', DATE) AS month,
             AVG(CLOSE_PRICE) AS avg_close
      FROM SNOWFLAKE_INTELIGENCE_HOL.DATA.STOCK_PRICE_HISTORY
      WHERE TICKER = 'NVDA'
        AND DATE >= DATE_TRUNC('year', CURRENT_DATE())
      GROUP BY 1
      ORDER BY 1

  - name: total_volume_last_30_days
    question: How many shares traded for AAPL in the last 30 days?
    sql: >
      SELECT SUM(VOLUME) AS total_volume
      FROM SNOWFLAKE_INTELIGENCE_HOL.DATA.STOCK_PRICE_HISTORY
      WHERE TICKER = 'AAPL'
        AND DATE >= DATEADD('day', -30, CURRENT_DATE())
  $$
  -- Omit third parameter or set to FALSE to create the view
);

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'SNOWFLAKE_DOCUMENTATION.SHARED.CKE_SNOWFLAKE_DOCS_SERVICE',
      '{
        "query": "how do i configure a default schema for a snowflake user account",
        "columns":[
            "chunk",
            "document_title",
            "source_url"
        ],
        "limit":10
      }'
  )
)['results'] as results
;



In [ ]:
 
result = cells.TEST_SEARCH_SQL.to_pandas()


In [ ]:
result

In [ ]:
df = pd.DataFrame(json.loads(result["RESULTS"][0]))
df

[Query Cortex Search REST API](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service#rest-api)

## Configure Cortex Agent

Now that we have all the pieces in place for the agent to pull context from, it's time to create the agent. The rest of the process will be done in Snowsight. Follow the steps below to complete the agent creation.


### Step-by-Step Guide
1. Create the Agent 
    - Go to AI & ML > Agents
    - Select "Create Agent"
    - Enter agent name and description
2. Add Cortex Search Service
    - Select "Tools" > "Cortex Search Services" > "+ Add"
    - Select your existing Cortex Search service
    - Enter chunk_id as ID column and document_name as title column.
    - Enter name and description for the search service
    - Click "Add"
3. Add Cortex Analyst Tool (Using Semantic Model)
    - Select "Tools" > "Cortex Analyst" > "+ Add"
    - Enter name for the semantic model tool
    - Select the semantic view
    - Choose warehouse for query execution
    - Set query timeout (seconds)
    - Add description
    - Click "Add"
4. Add Custom Tools
    - Select "Tools" > "Custom Tools" > "+ Add"
    - Leave resource type as procedure
    - Select existing stored procedure
    - Configure parameters (name, type, description, required status)
    - Select warehouse for execution (leave as default)
    - Add tool description and optionally argument descriptions
    - Click "Add"
5. Add Custom Prompt/Instructions
    - In the agent configuration, add custom instructions for:
        * Response behavior
        * Orchestration logic
        * System prompts
6. Save and Test
    - Click "Save" to create the agent
    - Test with sample questions
    - Monitor agent performance and user feedback